In [ ]:
#import libs
import numpy as np
import pandas as pd
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Імпортуємо отриманий файл з диска


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TEST TASK (dataset).csv')

In [ ]:
df.head()

,total_sessions_day0,total_sessions_day1,total_sessions_day3,total_sessions_day7,chapters_finished_day0,chapters_finished_day1,chapters_finished_day3,chapters_finished_day7,chapters_opened_day0,chapters_opened_day1,chapters_opened_day3,chapters_opened_day7,chapters_closed_day0,chapters_closed_day1,chapters_closed_day3,chapters_closed_day7,diamonds_received_day0,diamonds_received_day1,diamonds_received_day3,diamonds_received_day7,diamonds_spent_day0,diamonds_spent_day1,diamonds_spent_day3,diamonds_spent_day7,tickets_spent_day0,tickets_spent_day1,tickets_spent_day3,tickets_spent_day7,retained_day1,retained_day3,retained_day7,chapters_finished_session1,chapters_finished_session3,chapters_finished_session9,chapters_opened_session1,chapters_opened_session3,chapters_opened_session9,chapters_closed_session1,chapters_closed_session3,chapters_closed_session9,diamonds_spent_session1,diamonds_spent_session3,diamonds_spent_session9,tickets_spent_session1,tickets_spent_session3,tickets_spent_session9,app_sub_ltv_day0,app_sub_ltv_day1,app_sub_ltv_day3,app_iap_ltv_day0,app_iap_ltv_day1,app_iap_ltv_day3,media_source,install_date,country_code,ad_ltv_day0,ad_ltv_day1,ad_ltv_day3,platform,target_sub_ltv_day30,target_iap_ltv_day30,target_ad_ltv_day30,target_full_ltv_day30
0,1.0,1.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,SOURCE_23,2021-12-03,COUNTRY_135,0.0,0.0,0.0,ios,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,SOURCE_2,2021-12-03,COUNTRY_141,0.0,0.0,0.0,android,0.0,0.0,0.0,0.0
2,1.0,2.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,SOURCE_17,2021-12-03,COUNTRY_141,0.0,0.0,0.0,android,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,SOURCE_23,2021-12-03,COUNTRY_141,0.0,0.0,0.0,ios,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,SOURCE_1,2021-12-03,COUNTRY_203,0.0,0.0,0.0,android,0.0,0.0,0.0,0.0


Створимо новий датасет для прогнозування. Для цього прибираємо колонки, що дають нам лік даних при прогнозуванні. Також при проведенні EDA аналізу не було виявленно залежності від дати встановлення, тому видалимо її для облегшення нашого датасету.

In [ ]:
dfmodel = df
dfmodel = dfmodel.drop(columns=['target_ad_ltv_day30', 'target_iap_ltv_day30', 'target_iap_ltv_day30','install_date'])

Проводимо повторну перевірку 


In [ ]:
dfmodel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1467832 entries, 0 to 1467831
Data columns (total 60 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   total_sessions_day0         1466467 non-null  float64
 1   total_sessions_day1         1466465 non-null  float64
 2   total_sessions_day3         1466464 non-null  float64
 3   total_sessions_day7         1466458 non-null  float64
 4   chapters_finished_day0      1467832 non-null  int64  
 5   chapters_finished_day1      1467832 non-null  int64  
 6   chapters_finished_day3      1467832 non-null  int64  
 7   chapters_finished_day7      1467832 non-null  int64  
 8   chapters_opened_day0        1467832 non-null  int64  
 9   chapters_opened_day1        1467832 non-null  int64  
 10  chapters_opened_day3        1467832 non-null  int64  
 11  chapters_opened_day7        1467832 non-null  int64  
 12  chapters_closed_day0        1467832 non-null  int64  
 1

Отримали як результат перевірку значень та типу даних перемінних.

Проводимо енкодінг даних, для того щоб наша модель могла працювати з ними.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
dfmodel['media_source']=le.fit_transform(dfmodel['media_source'])
dfmodel['country_code']=le.fit_transform(dfmodel['country_code'])
dfmodel['platform']=le.fit_transform(dfmodel['platform'])
dfmodel.fillna(0, inplace=True)

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, KFold

/usr/local/lib/python3.7/dist-packages/lightgbm/__init__.py:30: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/lightgbm/VERSION.txt' mode='r' encoding='UTF-8'>
  __version__ = open(os.path.join(dir_path, 'VERSION.txt')).read().strip()


Прогнозувати ми будемо колонку target_full_ltv_day30.

Ділимо наші дані на тестову та тренувальну вибірки з поділом 15% та 85% відповідно.

In [ ]:
X = dfmodel.drop("target_full_ltv_day30",axis=1)
Y = dfmodel['target_full_ltv_day30']

In [ ]:
train, test, target_train, target_test = train_test_split(X, Y, test_size=0.15,random_state=42)

Тепер для покращення результатів роботи алгоритму в майбутньому поділимо тренувальний датасет на валідаційний з розділом на 75% та 25%.
Також введемо random_state=42 для того щоб уникнути неправильного розділення даних на тестову та тренувальну, щоб дані користувачів, що принесли прибуток, були і в тренувальних та в тестових вибірках.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target_train, test_size=0.25,random_state=42)

Проведемо нормалізацію даних для спрощення роботи алгоритма та покращення результатів. Застосовуємо функцію Pipeline. Вибираємо першу модель.

Проведемо первинний аналіз та порахуємо похибку RMSE для тренувальної та тестової вибірки.

In [ ]:
pipe = Pipeline([('scaler', MinMaxScaler()), ('model', LinearRegression())])
pipe.fit(X_train, y_train)

y_preds = pipe.predict(X_valid)
# replace zeros with negatives
y_preds = np.where(y_preds >  0, y_preds, 0)
mean_squared_error(y_valid, y_preds, squared=False)

1.0395351775900732

RMSE тестової вибірки.

In [ ]:
y_preds = pipe.predict(test)
y_preds = np.where(y_preds > 0, y_preds, 0)
mean_squared_error(target_test, y_preds, squared=False)

0.9301955165995409

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
MAPE = mean_absolute_percentage_error(target_test, y_preds)
RMSE = mean_squared_error(target_test, y_preds, squared=False)
MAE = mean_absolute_error(target_test, y_preds)

Основні метрики моделі:

In [ ]:
 
print(MAPE,RMSE,MAE)

45898893001743.305 0.9301955165995409 0.09656412591453746


Метрика МАРЕ в цьому випадку є не інформативною, бо при її підрахунку ми ділимо на значення таргету(а так як в нас багато нульових значень, то ми ділимо на нуль)

Будемо спиратися на RMSE та MAE.

RMSE на відміну від MAE може вказувати наскільки сильно відрізняються значення предикту від таргета(бо при підрахунку різниця значень береться у квадрат)

А МАЕ це абсолютна похибка, тому на неї теж варто спиратися при аналізі роботи моделі

Застосуєсо метод RandomForstRegressor.
Та знайдемо RMSE тренувальної вибірки.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
pipe = Pipeline([('scaler', MinMaxScaler()), ('model', regr)])
pipe.fit(X_train, y_train)

In [ ]:
y_preds = pipe.predict(X_valid)
# replace zeros with negatives
y_preds = np.where(y_preds > 0, y_preds, 0)
mean_squared_error(y_valid, y_preds, squared=False)

1.4211565511909998

RMSE тестової вибірки.

In [ ]:
y_preds = pipe.predict(test)
y_preds = np.where(y_preds > 0, y_preds, 0)
mean_squared_error(target_test, y_preds, squared=False)

1.2197580785366386

In [ ]:
MAPE = mean_absolute_percentage_error(target_test, y_preds)
RMSE = mean_squared_error(target_test, y_preds, squared=False)
MAE = mean_absolute_error(target_test, y_preds)

Основні метрики моделі:

In [ ]:
#RandomForest
print(MAPE,RMSE,MAE)

263194706306841.6 1.2197580785366386 0.20782880750751104


Застосуємо LGBM Model

In [ ]:
params = {
    'n_estimators': 400,
    'learning_rate': 0.075,
    'num_leaves': 255,
    'metric': 'mse'
}
model = LGBMRegressor(**params)

pipe = Pipeline([('scaler', MinMaxScaler()), ('model', model)])
pipe.fit(X_train, y_train);

RMSE тренувальної вибірки.

In [ ]:
y_preds = pipe.predict(X_valid)
y_preds = np.where(y_preds > 0, y_preds, 0)
mean_squared_error(y_valid, y_preds, squared=False)

0.9849865995634122

RMSE тестової вибірки.

In [ ]:
y_preds = pipe.predict(test)
y_preds = np.where(y_preds > 0, y_preds, 0)
mean_squared_error(target_test, y_preds, squared=False)

0.9593363842794088

In [ ]:
MAPE = mean_absolute_percentage_error(target_test, y_preds)
RMSE = mean_squared_error(target_test, y_preds, squared=False)
MAE = mean_absolute_error(target_test, y_preds)

Основні метрики моделі:

In [ ]:
#LGBMRegression
print(MAPE,RMSE,MAE)

5780091601190.708 0.9593363842794088 0.06332008851019137


При аналізі цих трьох моделей, фаворитами є LinearRegression(RMSE: 0.9301955165995409 MAE: 0.09656412591453746) та LGBM Regression(RMSE: 0.9593363842794088 MAE: 0.06332008851019137)

Абсолютна похибка у останньої моделі менша, тому спробуємо зробити кросвалідацію для метода LGBM Regression.

Зробимо модель з кросвалідацією на 5 фолдів, щоб знайти оптимальную вибірку для навчання моделі.

In [ ]:
params = {
    'n_estimators': 400,
    'learning_rate': 0.075,
    'num_leaves': 255,
    'metric': 'mse'
}
model = LGBMRegressor(**params)

pipe = Pipeline([('scaler', MinMaxScaler()), ('model', model)])

In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits)

valid_ans = np.zeros(y_valid.shape[0])
test_ans = np.zeros(target_test.shape[0])
current_fold = 0
for train_index, test_index in kf.split(X_train):
    current_fold += 1
    print("Fold #", current_fold, "initiate...")
    X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]
    pipe.fit(X_train_cv, y_train_cv)
    y_pred_fold = pipe.predict(X_test_cv)
    print("Fold MSE: ", mean_squared_error(y_test_cv, y_pred_fold))
    valid_ans += pipe.predict(X_valid)
    test_ans += pipe.predict(test)
print("Valid prediction: ", mean_squared_error(y_valid, valid_ans / n_splits))

Fold # 1 initiate...
Fold MSE:  0.966649623447414
Fold # 2 initiate...
Fold MSE:  0.9064367431307835
Fold # 3 initiate...
Fold MSE:  0.8309525689965284
Fold # 4 initiate...
Fold MSE:  0.7682935196162444
Fold # 5 initiate...
Fold MSE:  1.210517990715026
Valid prediction:  0.9614668887961874


Для подальшого прогнозування, на основі метрик було обрано дві моделі:
LGBM Regression та LinearRegression




Імпортуємо новий датасет, який ми очистили від аномальних даних.
Та готуємо його по аналогії з оригінальни датасетом для роботи з моделлю.

In [ ]:
dfNew = pd.read_csv('/content/drive/MyDrive/datasets/full_set.csv')

In [ ]:
dfNew = dfNew.drop(columns=['target_ad_ltv_day30', 'target_iap_ltv_day30', 'target_iap_ltv_day30','install_date'])

In [ ]:
dfNew['media_source']=le.fit_transform(dfNew['media_source'])
dfNew['country_code']=le.fit_transform(dfNew['country_code'])
dfNew['platform']=le.fit_transform(dfNew['platform'])

In [ ]:
X = dfNew.drop("target_full_ltv_day30",axis=1)
Y = dfNew['target_full_ltv_day30']

In [ ]:
train, test, target_train, target_test = train_test_split(X, Y, test_size=0.15,random_state=42)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target_train, test_size=0.25,random_state=42)

LGBM Regression:

In [ ]:
params = {
    'n_estimators': 400,
    'learning_rate': 0.075,
    'num_leaves': 255,
    'metric': 'mse'
}
model = LGBMRegressor(**params)

pipe = Pipeline([('scaler', MinMaxScaler()), ('model', model)])
pipe.fit(X_train, y_train);

RMSE тренувальної вибірки.

In [ ]:
y_preds = pipe.predict(X_valid)
y_preds = np.where(y_preds > 0, y_preds, 0)
mean_squared_error(y_valid, y_preds, squared=False)

0.8768522275536376

RMSE тестової вибірки.

In [ ]:
y_preds = pipe.predict(test)
y_preds = np.where(y_preds > 0, y_preds, 0)
mean_squared_error(target_test, y_preds, squared=False)

0.8197823716797542

In [ ]:
MAPE = mean_absolute_percentage_error(target_test, y_preds)
RMSE = mean_squared_error(target_test, y_preds, squared=False)
MAE = mean_absolute_error(target_test, y_preds)

Основні метрики моделі:

In [ ]:
 
print(MAPE,RMSE,MAE)

5165392316713.884 0.8197823716797542 0.0550964166303383


Побудуємо модель LinearRegression на новому датасеті та знайдемо RMSE тренувальної вибірки.


In [ ]:
pipe = Pipeline([('scaler', MinMaxScaler()), ('model', LinearRegression())])
pipe.fit(X_train, y_train)

y_preds = pipe.predict(X_valid)
# replace zeros with negatives
y_preds = np.where(y_preds >  0, y_preds, 0)
mean_squared_error(y_valid, y_preds, squared=False)

0.9064899605707684

RMSE тестової вибірки.

In [ ]:
y_preds = pipe.predict(test)
y_preds = np.where(y_preds > 0, y_preds, 0)
mean_squared_error(target_test, y_preds, squared=False)

0.8869972728909266

In [ ]:
MAPE = mean_absolute_percentage_error(target_test, y_preds)
RMSE = mean_squared_error(target_test, y_preds, squared=False)
MAE = mean_absolute_error(target_test, y_preds)

In [ ]:
 
print(MAPE,RMSE,MAE)

39734203900817.93 0.8869972728909266 0.08713222472480037


На опрацьованому датасеті результати роботи моделей значно покращилися.

LinearRegression(RMSE: 0.8869972728909266 MAE: 0.08713222472480037) 

та LGBM Regression(RMSE: 0.8197823716797542 MAE: 0.0550964166303383)
проти:

LinearRegression(RMSE: 0.9301955165995409 MAE: 0.09656412591453746) та

LGBM Regression(RMSE: 0.9593363842794088 MAE: 0.06332008851019137)
на оригінальному датасеті.

Це доводить, що гіпотеза висунунта нами під час EDA, що зміна датасета на 0.69% суттєво покращує результати роботи моделі та оптимизовує роботи алгоритму.

Найкраще себе показала модель LGBM Regression, тому зробимо кросвалідацію для неї.

Зробимо модель з кросвалідацією на 5 фолдів, щоб знайти оптимальную вибірку для навчання моделі.

In [ ]:
params = {
    'n_estimators': 400,
    'learning_rate': 0.075,
    'num_leaves': 255,
    'metric': 'mse'
}
model = LGBMRegressor(**params)

pipe = Pipeline([('scaler', MinMaxScaler()), ('model', model)])

In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits)

valid_ans = np.zeros(y_valid.shape[0])
test_ans = np.zeros(target_test.shape[0])
current_fold = 0
for train_index, test_index in kf.split(X_train):
    current_fold += 1
    print("Fold #", current_fold, "initiate...")
    X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]
    pipe.fit(X_train_cv, y_train_cv)
    y_pred_fold = pipe.predict(X_test_cv)
    print("Fold MSE: ", mean_squared_error(y_test_cv, y_pred_fold, squared=False))
    valid_ans += pipe.predict(X_valid)
    test_ans += pipe.predict(test)
print("Valid prediction: ", mean_squared_error(y_valid, valid_ans / n_splits))

Fold # 1 initiate...
Fold MSE:  0.781239118937473
Fold # 2 initiate...
Fold MSE:  0.8646904078752004
Fold # 3 initiate...
Fold MSE:  0.8383111013094852
Fold # 4 initiate...
Fold MSE:  0.9112444933841288
Fold # 5 initiate...
Fold MSE:  0.8805763697552573
Valid prediction:  0.7358943120103927


In [ ]:
y_preds = pipe.predict(test)
y_preds = np.where(y_preds > 0, y_preds, 0)
mean_squared_error(target_test, y_preds, squared=False)

0.806829211389713

In [ ]:
MAPE = mean_absolute_percentage_error(target_test, y_preds)
RMSE = mean_squared_error(target_test, y_preds, squared=False)
MAE = mean_absolute_error(target_test, y_preds)

###**Метрики**###


In [ ]:
 
print(MAPE,RMSE,MAE)

5474181046390.7705 0.806829211389713 0.056573250053303666


In [ ]:
import joblib
# save model
joblib.dump(model, 'lgb.pkl')

['lgb.pkl']

Метрика МАРЕ в цьому випадку є не інформативною, бо при її підрахунку ми ділимо на значення таргету(а так як в нас багато нульових значень, то ми ділимо на нуль)

RMSE ми вибираємо як основну, через її природу, оскільки вона гарно відпрацьовує на великих викидах, що підходить нам черз тип задачі.
MAE будемо використовувати як допоміжну метрику для аналізу якості моделі.

###**Важливість фіч**###

Провевши навчання моделі та отримавши перші результати ми можемо проаналізувати важливість "фіч" для прогнозування вибраних нами значень.

In [ ]:
importance_LGB = model.feature_importances_
for i,v in enumerate(importance_LGB):
    print('Feature: %0d, Score: %.6f' % (i, v))

Feature: 0, Score: 1449.000000
Feature: 1, Score: 1864.000000
Feature: 2, Score: 2320.000000
Feature: 3, Score: 3447.000000
Feature: 4, Score: 1012.000000
Feature: 5, Score: 984.000000
Feature: 6, Score: 1358.000000
Feature: 7, Score: 2326.000000
Feature: 8, Score: 919.000000
Feature: 9, Score: 967.000000
Feature: 10, Score: 1274.000000
Feature: 11, Score: 1874.000000
Feature: 12, Score: 608.000000
Feature: 13, Score: 719.000000
Feature: 14, Score: 1067.000000
Feature: 15, Score: 2060.000000
Feature: 16, Score: 2078.000000
Feature: 17, Score: 2286.000000
Feature: 18, Score: 3027.000000
Feature: 19, Score: 3572.000000
Feature: 20, Score: 1865.000000
Feature: 21, Score: 1992.000000
Feature: 22, Score: 2766.000000
Feature: 23, Score: 4816.000000
Feature: 24, Score: 1560.000000
Feature: 25, Score: 1155.000000
Feature: 26, Score: 1686.000000
Feature: 27, Score: 3499.000000
Feature: 28, Score: 249.000000
Feature: 29, Score: 272.000000
Feature: 30, Score: 682.000000
Feature: 31, Score: 556.00

З отриманих результатів можемо зробити висновки стосовно важливості зміних.
А саме 
Feature: 42, Score: 6186.000000, diamonds_spent_session9
Feature: 23, Score: 4816.000000, diamonds_spent_day7
Feature: 51, Score: 4200.000000, app_iap_ltv_day3
Feature: 19, Score: 3572.000000, diamonds_received_day7
Feature: 58, Score: 3505.000000, target_sub_ltv_day30
Feature: 27, Score: 3499.000000, tickets_spent_day7

Згадуючи значення змінних можемо сказати, що схожі дані ми отримали під час проведення EDA аналізу. Це показує що алгоритм правильно знаходить та показує взаємозвязки. Це доводить, що алгоритм був правильно натренований.

###**Висновки**###

Висновками цієї роботи ми можемо винести результати роботи нашого алгоритму, а саме

LGBM Regression з резудьтами роботи 
RMSE = 0.806829211389713
MAE = 0.056573250053303666

Також була підтверджена гіпотеза стосвно негативного впливу неправильнних даних та показана суттєва різниця результатів після обробки датасету.

Провели крос валідацію для вибору найкращого набору датасету.

Перевірили правильність роботи алгоритму за дпопомгою перевірки важливості фіч, де довели, що доходи від клієнта залежать в основному від кристалів. Також це було в висновках EDA аналізу.

Best regards, team Checkinst